In [9]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
from sklearn.preprocessing import LabelEncoder

In [10]:
data = pd.read_csv('Sentiment.csv')
# Keeping only the neccessary columns
data = data[['text','sentiment']]

In [11]:
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

In [12]:
for idx, row in data.iterrows():
    row[0] = row[0].replace('rt', ' ')

In [13]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)

In [14]:
X = pad_sequences(X)

In [15]:
embed_dim = 128
lstm_out = 196
def createmodel():
    model = Sequential()
    model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(3,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    return model

In [16]:
labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data['sentiment'])
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42)
print(labelencoder.classes_)
print(labelencoder.transform(labelencoder.classes_))

['Negative' 'Neutral' 'Positive']
[0 1 2]


In [17]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
batch_size = [16,32,64]
epochs = [1,2,3]
model = KerasClassifier(build_fn=createmodel,verbose=1)
param_grid= dict(batch_size=batch_size, epochs=epochs)
grid  = GridSearchCV(estimator=model, param_grid=param_grid)
grid_result= grid.fit(X_train, Y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

117/117 [==============================] - 1s 9ms/step - loss: 0.7748 - accuracy: 0.6728
Epoch 1/2
465/465 [==============================] - 34s 73ms/step - loss: 0.8290 - accuracy: 0.6426
Epoch 2/2
117/117 [==============================] - 1s 9ms/step - loss: 0.7497 - accuracy: 0.6697
Epoch 1/2
465/465 [==============================] - 36s 76ms/step - loss: 0.8285 - accuracy: 0.6457
Epoch 2/2
117/117 [==============================] - 1s 7ms/step - loss: 0.7521 - accuracy: 0.6826
Epoch 1/2
465/465 [==============================] - 35s 75ms/step - loss: 0.8283 - accuracy: 0.6419
Epoch 2/2
117/117 [==============================] - 1s 9ms/step - loss: 0.7383 - accuracy: 0.6805
Epoch 1/2
465/465 [==============================] - 34s 74ms/step - loss: 0.8315 - accuracy: 0.6403
Epoch 2/2
117/117 [==============================] - 1s 10ms/step - loss: 0.7618 - accuracy: 0.6744
Epoch 1/2
465/465 [==============================] - 35s 75ms/step - loss: 0.8353 - accuracy: 0.6398
Epoch 2/2

59/59 [==============================] - 2s 30ms/step - loss: 0.7713 - accuracy: 0.6835
Epoch 1/3
233/233 [==============================] - 40s 171ms/step - loss: 0.8380 - accuracy: 0.6405
Epoch 2/3
233/233 [==============================] - 37s 158ms/step - loss: 0.6707 - accuracy: 0.7212
Epoch 3/3
30/30 [==============================] - 1s 22ms/step - loss: 0.7822 - accuracy: 0.6652
Epoch 1/2
117/117 [==============================] - 14s 121ms/step - loss: 0.8703 - accuracy: 0.6275
Epoch 2/2
30/30 [==============================] - 1s 19ms/step - loss: 0.7483 - accuracy: 0.6713
Epoch 1/2
117/117 [==============================] - 18s 154ms/step - loss: 0.8595 - accuracy: 0.6268
Epoch 2/2
30/30 [==============================] - 1s 19ms/step - loss: 0.7384 - accuracy: 0.6783
Epoch 1/2
117/117 [==============================] - 15s 131ms/step - loss: 0.8816 - accuracy: 0.6154
Epoch 2/2
30/30 [==============================] - 1s 27ms/step - loss: 0.8126 - accuracy: 0.6407
Epoch 1/2


In [25]:
best_model = createmodel()
best_model.fit(X_train, Y_train, epochs = 2, batch_size=32, verbose = 1)
score,acc = best_model.evaluate(X_test,Y_test,verbose=2,batch_size=32)
print(best_model.summary())
print(score)
print(acc)
print(best_model.metrics_names)

Epoch 1/2
291/291 [==============================] - 23s 79ms/step - loss: 0.8263 - accuracy: 0.6395
Epoch 2/2
291/291 [==============================] - 33s 113ms/step - loss: 0.6742 - accuracy: 0.7117
144/144 - 4s - loss: 0.7397 - accuracy: 0.6800
Model: "sequential_51"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_51 (Embedding)     (None, 28, 128)           256000    
_________________________________________________________________
lstm_51 (LSTM)               (None, 196)               254800    
_________________________________________________________________
dense_51 (Dense)             (None, 3)                 591       
Total params: 511,391
Trainable params: 511,391
Non-trainable params: 0
_________________________________________________________________
None
0.7396810054779053
0.6799912452697754
['loss', 'accuracy']


In [27]:
best_model.save("sentimentAnalysis_grid.h5")

In [32]:
print(best_model.predict_classes(X_test[[5],:]))

[0]


In [33]:
print(Y_test[5])

[1. 0. 0.]
